# Herramientas complementarias para el manejo de bases de datos

En nuestro encuentro anterior, revisamos algunas cuestiones genéricas sobre POO. Vimos lo que es un método y las distintas formas de aplicarlo. También introducimos algunas cuestiones más iniciáticas como las estructuras nativas de python para trabajar con colecciones, los distintos tipos de datos y las operaciones disponibles tanto por defecto como a partir de la introducción de librerías como numpy y pandas. 

Ahora, intentaremos integrar esto con algunos de los conceptos que estuvimos viendo con anterioridad. Todo, con la finalidad de poder ampliar la bateria de recursos para trabajar con esquemas tabulares.


In [ ]:
# empecemos por importar pandas con su clásico alias
import pandas as pd

### Compañeros de ruta: la función lambda y los métodos map() y apply() 

Ahora que ya sabemos cuáles son los conceptos centrales de esta librería, vamos a necesitar empezar a manipular nuestros datos. Agregar o transformar columnas en un dataframe, recorrer filas para hacer algún tipo de transformación o, incluso calcular nuevas variables. 

Lo que vamos a hacer ahora, es repasar algunas formas de hacer este tipo de cosas. Fundamentalmente, retomando algo de lo que vimos últimamente: métodos y funciones. 

Hagamos una introducción breve para conocer las estrategias más comunes, cómo se usan y cuál es su equivalencia con estructuras de datos nativas de python. 

Hicimos esta selección porque creemos que son las de uso más frecuente y te servirán para terminar de redondear algunas cuestiones claves.

> **1. La función anónima `lambda`**

Es muy útil pensar esta función como si el **`lambda`** equivaliera al **`def`** en una función regular. Para ponerlo en otros términos, digamos que la manera en la que definimos una función anónima también nos permite trabajar con parámetros que vamos a usar para aplicar una expresión a partir de la que transformaremos algún objeto. 

<figure>
<center>
<img src='https://drive.google.com/uc?id=1576koLo1eMwbVT9rKQmM_aypzBgLSFs4' />
<figcaption></figcaption></center>
</figure>


Esto, con dos diferencias esenciales. **La primera, que la función anónima no requiere de un `return` para devolvernos un resultado**. 

<figure>
<center>
<img src='https://drive.google.com/uc?id=1q8NCAy15JLtYN7XYYmuXxS-ShzpETUU9' />
<figcaption></figcaption></center>
</figure>


Veamos algún ejemplo concreto. Para ello, supongamos que contamos con un código (algo bastante común para reconocer unidades administrativas o físicas en un territorio dado) y que queremos hacer alguna transformación sobre el mismo. Por ejemplo, completar su formato para contar con mayor información o hacer algún matcheo o unión con fuentes de información externas. Imaginemos que estamos trabajando con el código postal y que sólo contamos con una de las siguientes secciones...

Vamos un caso real, el del CPA. El mismo está compuesto por ocho caracteres:
1. Una letra identificatoria de la provincia.
2. Un número de 4 dígitos que identifica la localidad, ciudad o barrio.
3. Una combinación de tres letras que identifican la "cara" de la manzana.

<figure>
<center>
<img src='https://drive.google.com/uc?id=1GKJK8o8IGzOGQF_siWhOLAg6lSAYIAbV' />
<figcaption></figcaption></center>
</figure>


¿Cómo podríamos completarlo con otras secciones utilizando una función anónima si sólo contaramos con la intermedia, por ejemplo? Veamos cómo...

In [ ]:
# instanciamos nuestro código incompleto.
codigo_postal = '1024'

In [ ]:
# definimos una función para formar códigos postales que estén en una misma jurisdicción y cara de manzana
completar_formato = lambda x: 'C'+ x +'FDA'

In [ ]:
# la aplicamos
completar_formato(codigo_postal)

'C1024FDA'

In [ ]:
def completa_formato_regular(x):
  return 'C' + x + 'FDA'

In [ ]:
completa_formato_regular(codigo_postal)

'C1024FDA'

Como pudimos ver, `x` es nuestro parámetro, aquel sobre el cual se aplicará la expresión delimitada a partir del `lambda`.

**La segunda diferencia, es que las funciones anónimas sólo trabajan con una expresión en su cuerpo** (mientras que las regulares pueden contener muchas). `x` (o como nosotros decidamos que se llame) es lo que se conoce como un `place holder`. Es decir, podemos aplicar la expresión definida en la función a filas de una serie de pandas, a items de una lista, etc. Ahora bien, la función lambda sólo toma una expresión a la vez. Es decir, que debemos aplicarla a tantos objetos o elementos como parámetros hayamos definido. Veamos qué pasa si no respetamos esto:

In [ ]:
# definimos nuevamente nuestro código
codigo_postal = ['C','1024','FDA']

# rehacemos la función anónima, ahora agregando algunos parámetros más después de la key 'lambda'
completar_formato = lambda x,y,z: x + y + z

In [ ]:
# prestemos atención al tipo de error que nos devuelve
completar_formato(codigo_postal)

TypeError: ignored

In [ ]:
# le estoy pasando un solo elemento
codigo_postal[0]+codigo_postal[1]+codigo_postal[2]

'C1024FDA'

Como se puede leer, hay dos argumentos que aparecen como missing. Y esto es porque estamos aplicando una función con tres parametros sobre un solo objeto - una lista. Distinto hubiese sido si...

In [ ]:
# instanciamos por separado las distintas parte del código
parte_uno, parte_dos, parte_tres = 'C','1024','FDA'

# y aplicamos la función sobre las tres partes
completar_formato(parte_uno, parte_dos, parte_tres)

'C1024FDA'

O si...

In [ ]:
# suponiendo que tenemos una serie con muchas secciones de codigos postales ordenadas
codigo_postal = pd.Series(['C','1024','FDA',
                           'C','1171','ABM',
                           'C','1097','AAX'])

In [ ]:
codigo_postal

0       C
1    1024
2     FDA
3       C
4    1171
5     ABM
6       C
7    1097
8     AAX
dtype: object

In [ ]:
#list(codigo_postal.index)

In [ ]:
# rearmamos nuestra función lambda con un solo parametro
completar_formato = lambda x: [x[i]+x[i+1]+x[i+2] for i in x.index if x[i] == 'C']

In [ ]:
completar_formato(codigo_postal)

['C1024FDA', 'C1171ABM', 'C1097AAX']

In [ ]:
# o en lugar de filtrar por el tipo de letra, por su largo. Siempre sabemos que la primera es C
completar_formato = lambda x:[x[i]+x[i+1]+x[i+2] for i in x.index if len(x[i])==1]

In [ ]:
completar_formato(codigo_postal)

['C1024FDA', 'C1171ABM', 'C1097AAX']

In [ ]:
for i in codigo_postal.index:
  print(i)

0
1
2
3
4
5
6
7
8


In [ ]:
codigo_postal[0]

'C'

In [ ]:
# para obtener todos los codigos postales con las secciones que estaban dispersas, ahora concatendas
pd.Series(completar_formato(codigo_postal))

0    C1024FDA
1    C1171ABM
2    C1097AAX
dtype: object

> **2. El método `map`**

Este método se utiliza para aplicar una función a todos los elementos de un iterable especificado. De esta manera, su sintáxis queda definida como:

<figure>
<center>
<img src='https://drive.google.com/uc?id=1XeV5x3Vqvq-cGVvYS4B4SVux_Nl03xDs' />
<figcaption></figcaption></center>
</figure>


Se pueden pasar múltiples argumentos dentro de un `map`, siempre que la función que se esté mapeando tenga esa misma cantidad de argumentos previamente definidos. Veamos algunos ejemplos...

In [ ]:
# instanciamos por separado las distintas parte del código
parte_uno, parte_dos, parte_tres = 'C', '1024', 'FDA'
#parte_uno, parte_dos, parte_tres = ['C','C','C'],['1024','1171','1097'],['FDA','ABM','AAX']

In [ ]:
# armamos una funcion regular que sume tres variables
def combina(a,b,c):
    return a+b+c

In [ ]:
# aplicamos la función a las tres secciones de nuestro código postal
list(map(combina,parte_uno, parte_dos, parte_tres))

['C1F']

Detengámonos brevemente en qué nos devuelve, un obtejo `map`. Este es un [`iterador`](https://docs.python.org/3/glossary.html#term-iterator) y en python está diseñado así por una cuestión de eficiencia. Es decir, para evitar guardar en memoria todo sobre lo que se itera, map devuelve un objeto de tipo iterador que se puede visualizar cuando le aplicamos algún contenedor, como una lista o una serie. 

In [ ]:
list(map(combina,[parte_uno], [parte_dos], [parte_tres]))

['C1024FDA']

In [ ]:
pd.Series(map(combina,[parte_uno],[parte_dos], [parte_tres]))

0    C1024FDA
dtype: object

Y por qué se puede leer cuando le aplicamos un contenedor?, basicamente porque el objeto de tipo `map` es un objeto iterable, es decir, que se puede iterar sobre él:

In [ ]:
for i in map(combina, [parte_uno], [parte_dos], [parte_tres]):
    print(i)

C1024FDA


Esto significa que, por ejemplo, si aplcaramos un `list()` a un `map(func, iter)` estaría sucediendo algo como esto:

In [ ]:
codigo = []
for i in map(combina,[parte_uno], [parte_dos], [parte_tres]):
    codigo.append(i)
    
codigo

['C1024FDA']

Este comportamiento hace que `map()` sea más rápido que otras alternativas, por ejemplo una lista por comprensión.

In [ ]:
partes = ['C','1024','FDA','C','1171','ABM','C','1097','AAX']

In [ ]:
[i for i in range(len(partes))]

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [ ]:
# lista por comprension
resultado = [(partes[i]+partes[i+1]+partes[i+2]) for i in range(len(partes)) if partes[i]=='C']

In [ ]:
resultado

['C1024FDA', 'C1171ABM', 'C1097AAX']

In [ ]:
# esto es lo que está sucediendo cuando aplicamos una lista por comprensión
codigos = []

for i in range(len(partes)):
    if (partes[i]=='C'):
        codigos.append(partes[i]+partes[i+1]+partes[i+2])

In [ ]:
codigos

['C1024FDA', 'C1171ABM', 'C1097AAX']

Si esto mismo lo hacemos utilizando el método `map`, primero debemos crear una función que realice la misma tarea. Recordemos, que este itera sobre cada uno de los elementos de un contenedor (en nuestro ejemplo previo, una lista). Por lo tanto, deberemos adaptar o bien la forma en la que diseñamos nuestra función o bien el objeto sobre el que la aplicamos. Hagamos esto último para  no extendernos demasiado...

In [ ]:
def combina_partes(lista):
    '''
    Combina los items de un objeto iterable.
    ...
    Argumentos:
        arg(iterable): iterable de strings 
                       (e.g. 'list' o 'pandas serie')
    Devuelve:
        lista: items combinados
    '''
    codigos=[]
    for i in range(len(lista)):
        if (lista[i]=='C'): # identificamos el caracter inicial del codigo
            codigos.append(lista[i]+lista[i+1]+lista[i+2])
    return codigos

Reparemos ahora en cómo la vamos a aplicar. Como nuestra función fue pensada para operar sobre una lista como único argumento, no podríamos mapear directamente sobre la lista `partes`. Por qué? básicamente porque sino dicha función se aplicaría sobre cada elemento `lista[0]`, `lista[1]`, `lista[n]`. Entonces la vamos a convertir en lista, para que el primer elemento que mapee el iterador sea nuestra lista de string con las secciones de distintos códigos postales.

In [ ]:
[partes]

[['C', '1024', 'FDA', 'C', '1171', 'ABM', 'C', '1097', 'AAX']]

In [ ]:
# mapeamos la función y accedemos al index 0 del resultado para tener nuestros códigos unificados en un solo lugar
resultado = list(map(combina_partes,[partes]))

In [ ]:
resultado

[['C1024FDA', 'C1171ABM', 'C1097AAX']]

In [ ]:
resultado[0]

['C1024FDA', 'C1171ABM', 'C1097AAX']

Como vimos hasta acá, el `map` aplica una función a un iterable y devuelve el resultado de haber aplicado dicha función en cada uno de los items. Esto es particularmente útil para realizar operaciones con columnas de un dataframe. Acá es donde el método `map` adquiere una mayor sentido para nosotros. 

In [ ]:
import numpy as np

In [ ]:
# veamos un ejemplo rápido de cómo mapear una función sobre una serie de valores aleatorios
columna = pd.Series(np.random.choice(100,10))

In [ ]:
columna

0    45
1    50
2    78
3    87
4    82
5    77
6    64
7    38
8    15
9    52
dtype: int64

¿Y cuál es la forma de aplicar una modificación que mejor performa?

In [ ]:
%%time
[round(i/2,1) for i in columna]

CPU times: user 84 µs, sys: 0 ns, total: 84 µs
Wall time: 91.1 µs


[22.5, 25.0, 39.0, 43.5, 41.0, 38.5, 32.0, 19.0, 7.5, 26.0]

In [ ]:
def num (n) :
    return round(n / 2,1)

In [ ]:
%%time
map(num, columna)

CPU times: user 34 µs, sys: 0 ns, total: 34 µs
Wall time: 38.1 µs


In [ ]:
%%time
map(lambda x: round(x/2,2), columna)

CPU times: user 29 µs, sys: 5 µs, total: 34 µs
Wall time: 38.1 µs


Pero digamos también que esto depende del contexto de uso. Una forma bastante frecuente de aplicar una función a una serie de pandas es cuando la utilizamos como si esta fuera un método del objeto. Así:

In [ ]:
# Esta aplicación puede ser menos performante
%%time
columna.map(lambda x: round(x/2,1))

CPU times: user 525 µs, sys: 0 ns, total: 525 µs
Wall time: 534 µs


0    22.5
1    25.0
2    39.0
3    43.5
4    41.0
5    38.5
6    32.0
7    19.0
8     7.5
9    26.0
dtype: float64

### BONUS: Simulando un contexto de uso

Pongamos este ejemplo un poco más en contexto para terminar de entender la utilidad del `map`. Supongamos que en la Ciudad de Buenos Aires se implementó un plan urbano que dictamina la demolición de edificios abandonados por un período de tiempo superior a los 20 años. Y supongamos también que los siguientes códigos postales:

In [ ]:
pd.Series(resultado)

0    [C1024FDA, C1171ABM, C1097AAX]
dtype: object

... corresponden a edificios que cumplen con dicha condición. Dada la crisis habitacional de la ciudad, para estos casos el nuevo plan urbano, dispone la demolición y posterior construcción de nuevas unidades de habitación que se vuelquen al mercado para aumentar la oferta de vivienda asequible. Para terminar de entender en qué nos puede ayudar el método map, digamos que en estos tres edificios se dispone la construcción de: 

In [ ]:
print("%s nuevos departamentos que estarán disponibles en el lapso de 24 meses" % (columna.sum()))

588 nuevos departamentos que estarán disponibles en el lapso de 24 meses


In [ ]:
resultado

[['C1024FDA', 'C1171ABM', 'C1097AAX']]

In [ ]:
columna[:3]

0    45
1    50
2    78
dtype: int64

In [ ]:
# veamos nuestra tabla inicial...
desarrollos = pd.DataFrame({'id':resultado[0], 
                           'inicio': [2020,2020,2020], 
                           'unidades': columna[:3], #usemos solo los primeros tres casos
                           'duracion':[24,24,24]})

In [ ]:
desarrollos

,id,inicio,unidades,duracion
0,C1024FDA,2020,45,24
1,C1171ABM,2020,50,24
2,C1097AAX,2020,78,24


Ahora, como también sabemos que el gobierno irá construyendo la misma cantidad de unidades por año, queremos que nuestra tabla de desarrollos adopte un nuevo formato. En este debe quedar representado, además del año de inicio el de finalización y la cantidad de unidades por año.

In [ ]:
# Armemos una función que haga lo que indica nuestro hipotético escenario...
def desarrollos_anuales(df, duracion, unidades, idx):
    '''
    Distribuye unidades residenciales a lo largo de
    los años de construcción del proyecto.
    ...
    Argumentos:
        df(dataframe): dataframe de proyectos residenciales 
        duracion(str): nombre de columna
        unidades(str): nombre de columna
        idx(str): nombre de columna
    Devuelve:
        dataframe: unidades anualmente distribuidas
    '''
    
    df['años'] = df[duracion].map(lambda x: x/12) # la cantidad de años del proyecto 
    df['anuales'] = df[unidades]/df['años'] # la cantidad de unidades por año

    new_df = np.repeat(df[idx], df['años']).reset_index() #creamos un nuevo df, con tantas filas
                                                          #como idx(o codigos postales) repetidos por año
    
    new_df['inicio'] = 2020 # seteamos el año base o de inicio

    gb = new_df.groupby(idx).size().map(range) #creamos un rango por codigo postal, con la cantidad de veces
                                               #que se repite a lo largo de las filas   

    itera_rango = lambda x: [i for i in x] #con esta funcion iteramos el rango que creamos previamente
    
    rango_x_idx = []
    for rango in gb:
      rango_x_idx.append(itera_rango(rango)) #obtenemos una lista de listas con los integer
                                             #resultantes de iterar por un range

    lista_aplanada = []
    for item in rango_x_idx:
      for subitem in item:
        lista_aplanada.append(subitem)  #esta es una manera larga, pero mas clara de ver
                                        # aca, estamos aplanando las listas de una sola.

    new_df['inicia'] = new_df['inicio'] + pd.Series(lista_aplanada) # indicamos el año de inicio    
    new_df['finaliza'] = new_df['inicia'] + 1 # y el de finalizacion
    
    oferta = np.repeat(df['anuales'],
                       df['años']).values #repetimos los valores de oferta anual tantos
                                          #años como tiene el proyecto

    new_df['unidades'] = oferta #creamos en el df la columna de unidades construidas por año
    
    new_df.drop(columns='inicio',inplace=True)
    
    return new_df.iloc[:,1:]

In [ ]:
desarrollos_anuales(desarrollos, 'duracion', 'unidades','id')

,id,inicia,finaliza,unidades
0,C1024FDA,2020,2021,22.5
1,C1024FDA,2021,2022,22.5
2,C1171ABM,2020,2021,25.0
3,C1171ABM,2021,2022,25.0
4,C1097AAX,2020,2021,39.0
5,C1097AAX,2021,2022,39.0


> **3. El método `apply`**

<figure>
<center>
<img src='https://drive.google.com/uc?id=1wY1g5EcNlsZGTi-EbDcFTWGPNGQuNVdU' />
<figcaption></figcaption></center>
</figure>




Siguiendo con el ejemplo anterior, supongamos que ahora necesitamos redondear el stock de unidades que se van a construir. Es decir, necesitamos que nuestras unidades estén expresadas en enteros y no perder información cuando el redondeo elimina los decimales.

In [ ]:
da = desarrollos_anuales(desarrollos, 'duracion', 'unidades','id')

In [ ]:
da

,id,inicia,finaliza,unidades
0,C1024FDA,2020,2021,22.5
1,C1024FDA,2021,2022,22.5
2,C1171ABM,2020,2021,25.0
3,C1171ABM,2021,2022,25.0
4,C1097AAX,2020,2021,39.0
5,C1097AAX,2021,2022,39.0


In [ ]:
def entero(x):
    return int(x)

def resto(x):
    if float(x)-int(x)>0:
        return int(x)+1
    else:
        return int(x)+0

In [ ]:
def redondea_unidades(df,col1):
    '''
    Asigna el resto de la división al último año
    de construcción (como entero 1).
    ...
    Argumentos:
        df(dataframe): dataframe de proyectos residenciales 
        col1(str): nombre de columna
    Devuelve:
        lista: lista de int
    
    '''
    
    df['idx'] = df.index # creamos un indice para cada año de proyecto
    ultimo_año_idx = df.groupby('id').idx.last().values #guardamos el idx del ultimo año

    valores = []
    for i in df[col1].index: #iteramos por el indice de la columa que queremos redondear
      if i in ultimo_año_idx: #si este esta en el ultimo año
        valores.append(df.loc[df['idx']==i, col1].apply(resto).values[0]) #sumamos la unidad adicional del resto
      else:
        valores.append(df.loc[df['idx']==i, col1].apply(entero).values[0]) #sino nos quedamos con el entero

    return valores

In [ ]:
redondea_unidades(da,'unidades')

[22, 23, 25, 25, 39, 39]

In [ ]:
da['total'] = redondea_unidades(da,'unidades')

In [ ]:
da

,id,inicia,finaliza,unidades,idx,total
0,C1024FDA,2020,2021,22.5,0,22
1,C1024FDA,2021,2022,22.5,1,23
2,C1171ABM,2020,2021,25.0,2,25
3,C1171ABM,2021,2022,25.0,3,25
4,C1097AAX,2020,2021,39.0,4,39
5,C1097AAX,2021,2022,39.0,5,39


De esta forma, vemos que para los casos en los que las unidades cuentan con valores decimales, las funciones que aplicamos recurriendo al método `apply` nos permitieron asignar ese resto al último año de la construcción. Mientras que para aquellos casos en los que el resto era cero no se agregó ninguna unidad adicional.